Beat-n game with 10 pt limit

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const MXV = 9
const MNV = 1

1

In [4]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [5]:


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 10 and filter nondominated

In [7]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [8]:
lib0 = get_library(10)
lib0

126-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 1, 5, 999)
 Spud("", 1, 1, 1, 1, 2, 4, 999)
 Spud("", 1, 1, 1, 1, 3, 3, 999)
 Spud("", 1, 1, 1, 1, 4, 2, 999)
 Spud("", 1, 1, 1, 1, 5, 1, 999)
 Spud("", 1, 1, 1, 2, 1, 4, 999)
 Spud("", 1, 1, 1, 2, 2, 3, 999)
 Spud("", 1, 1, 1, 2, 3, 2, 999)
 Spud("", 1, 1, 1, 2, 4, 1, 999)
 Spud("", 1, 1, 1, 3, 1, 3, 999)
 Spud("", 1, 1, 1, 3, 2, 2, 999)
 Spud("", 1, 1, 1, 3, 3, 1, 999)
 Spud("", 1, 1, 1, 4, 1, 2, 999)
 ⋮
 Spud("", 3, 1, 3, 1, 1, 1, 999)
 Spud("", 3, 2, 1, 1, 1, 2, 999)
 Spud("", 3, 2, 1, 1, 2, 1, 999)
 Spud("", 3, 2, 1, 2, 1, 1, 999)
 Spud("", 3, 2, 2, 1, 1, 1, 999)
 Spud("", 3, 3, 1, 1, 1, 1, 999)
 Spud("", 4, 1, 1, 1, 1, 2, 999)
 Spud("", 4, 1, 1, 1, 2, 1, 999)
 Spud("", 4, 1, 1, 2, 1, 1, 999)
 Spud("", 4, 1, 2, 1, 1, 1, 999)
 Spud("", 4, 2, 1, 1, 1, 1, 999)
 Spud("", 5, 1, 1, 1, 1, 1, 999)

In [9]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [10]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [11]:
lib=filter_nondominated2(lib0)
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end
nlib = length(lib)

 111124 111133 111142 111214 111223 111232 111241 111313 111322 111331 111412 111421 111511 112114 112123 112132 112141 112213 112222 112231 112312 112321 112411 113113 113122 113131 113212 113221 113311 114112 114121 114211 121114 121123 121132 121213 121222 121231 121312 121321 121411 122113 122122 122212 122221 122311 123112 123211 131113 131122 131212 131221 131311 132112 132211 141112 141211 212113 212122 212131 212212 212221 212311 213112 213121 213211 214111 222112 222211 312112 312121 312211 313111 412111

74

In [12]:
payoffs = get_payoffs(lib)

74×74 Matrix{Int64}:
  0  -1  -1  -1  -1  -1  -1   1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   0  -1  -1  -1  -1  -1  -1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   0  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   0  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1  -1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1   1   1  -1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1  -1   1   1  -1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1   1  -1  -1   1   1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  ⋮ 

In [13]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

spud2int (generic function with 1 method)

In [14]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));
int2ind = Dict(spud2int(lib[i]) => i for i in 1:length(lib));

In [15]:
libi = [spud2int(ff) for ff in lib];

In [16]:
int2ind[313111]

73

In [17]:
as = [1,2,3]
as[end]

3

In [18]:
function eval_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    if length(bs)==0 || length(as)==0
        return 0
    end
    a = as[end]
    flag = true
    count = 0
    while flag
        if payoffs[a, bs[end-count]]==1
            count += 1
        else
            flag = false
        end
        if count == length(bs)
            flag = false
        end
    end
    return count
end

eval_beatn (generic function with 1 method)

In [19]:
as = [int2ind[ff] for ff in [214111, 112222, 313111, 113311]]
bs = [int2ind[ff] for ff in [121411, 121321, 113212, 213121]]

4-element Vector{Int64}:
 41
 40
 27
 65

In [20]:
eval_beatn(payoffs, as[1:3], bs[1:2])

2

In [21]:
eval_beatn(payoffs, bs[1:3], as[1:3])

2

In [22]:
eval_beatn(payoffs, as[1:4], bs[1:3])

2

In [23]:
eval_beatn(payoffs, bs[1:4], as[1:4])

3

In [24]:
size(payoffs)[1]

74

In [25]:
# tries all possible moves as extension of as
function try_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    ans = Array{Int64}(undef, size(payoffs)[1])
    for i in 1:size(payoffs)[1]
        as2 = copy(as)
        append!(as2, i)
        ans[i] = eval_beatn(payoffs, as2, bs)
    end
    return ans
end

try_beatn (generic function with 1 method)

In [26]:
for i in try_beatn(payoffs, as, bs)
    print(i)
end

00000000000000000000000011120110021211220011100001110000000000000120000000

In [27]:
# returns 1 if Player A lost
function lose_or_not(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    if maximum(scores) < prev_score
        return 1
    end
    return 0
end

lose_or_not (generic function with 1 method)

In [28]:
lose_or_not(payoffs, as[1:2], bs[1:2])

0

In [29]:
lose_or_not(payoffs, as[1:3], bs[1:3])

0

In [30]:
lose_or_not(payoffs, as[1:4], bs[1:4])

1

In [31]:
function legal_moves(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return cands
end

legal_moves (generic function with 1 method)

In [32]:
# picks randomly
function choose_random(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return rand(cands)
end

choose_random (generic function with 1 method)

In [33]:
# picks aggressively
function choose_aggro(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .== maximum(scores))
    return rand(cands)
end

choose_aggro (generic function with 1 method)

In [34]:
function play_random_vs_aggro(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_aggro(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function display_game(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})
    for i in 1:length(as)
        print(libi[as[i]])
        print(" {")
        n = eval_beatn(payoffs, as[1:i], bs[1:(i-1)])
        print(n)
        print("}. ")
        if i <= length(bs)
            print(libi[bs[i]])
            print(" {")
            n = eval_beatn(payoffs, bs[1:i], as[1:i])
            print(n)
            print("}. ")
        end
        println()
    end
end

display_game (generic function with 1 method)

In [35]:
a_win, as, bs = play_random_vs_aggro(payoffs, [int2ind[111133]], [int2ind[222211]])
display_game(payoffs, as, bs)

111133 {0}. 222211 {1}. 
213112 {1}. 113131 {2}. 
113221 {2}. 214111 {3}. 
122311 {3}. 121222 {3}. 
131221 {3}. 113311 {4}. 
121312 {5}. 


In [36]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# for a in 1:nlib
#     for b in 1:nlib
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a], [b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [37]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)
#     js = findall(wins[i, :].==minimum(wins[i, :]))
#     print(libi[i])
#     print(" ")
#     print(libi[js])
#     print(" ")
#     println(minimum(wins[i, :])/n_games)
# end

In [38]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a_moves = legal_moves(payoffs, [a1], [b1])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1], [a1, a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a], [b1,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [39]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [40]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2],[b1,b2])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
113212 {1}. 113311 {3}. 
131212 {3}. 113221 {3}. 
114112 {4}. 131311 {4}. 
131122 {5}. 121312 {5}. 


In [41]:
# n_games = 1000
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a2 = int2ind[213112]
# b2 = int2ind[313111]
# a_moves = legal_moves(payoffs, [a1,a2], [b1,b2])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1,b2], [a1,a2,a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a], [b1,b2,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [42]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [43]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3],[b1,b2,b3])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
141211 {0}. 113311 {4}. 


In [44]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a2 = int2ind[213112]
# b2 = int2ind[313111]
# a3 = int2ind[121321]
# b3 = int2ind[121222]
# a_moves = legal_moves(payoffs, [a1,a2,a3], [b1,b2,b3])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1,b2,b3], [a1,a2,a3,a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a], [b1,b2,b3,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [45]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [46]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a4 = int2ind[113122]
b4 = int2ind[111133]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 111133 {2}. 
212113 {2}. 113221 {2}. 
212131 {2}. 114112 {3}. 
111232 {4}. 113212 {4}. 
111313 {5}. 214111 {5}. 


## MCTS

In [47]:
function intarr_to_string(as::Array{Int64})::String
    return join([convert(Char, a+32) for a in as])
end

function string_to_intarr(s::String)::Array{Int64}
    return [convert(Int64, s[i])-32 for i in 1:length(s)]
end

# s=intarr_to_string([i for i in 1:75])
# as=string_to_intarr(s)
# for i in as
#     print(i)
#     print(" ")
# end

function split_arrs(zs::Array{Int64})::Tuple{Array{Int64},Array{Int64}}
    if mod(length(zs), 2)==0
        return (zs[1:2:(end-1)], zs[2:2:end])
    else
        return (zs[1:2:end], zs[2:2:(end-1)])
    end
end

function merge_arrs(as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    zs = Array{Int64}(undef, length(as)+length(bs))
    if mod(length(zs), 2)==0
        zs[1:2:(end-1)]=as
        zs[2:2:end]=bs
    else
        zs[1:2:end] = as
    zs[2:2:(end-1)] = bs
    end
    return zs
end

merge_arrs (generic function with 1 method)

In [48]:
function expand_leaf(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    # create nodes for all children if needed
    children = get_children(payoffs, leaf)
    for child in children
        if !haskey(tree, child)
            tree[child] = [0,0]
        end
    end
    # get arrays for leaf
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate random successor
    if player==1
        cands = legal_moves(payoffs, as, bs)
        append!(as, rand(cands))
    else
        cands = legal_moves(payoffs, bs, as)
        append!(bs, rand(cands))
    end    
    zs = merge_arrs(as, bs)
    # play the match
    a_win, as2, bs2 = play_random_vs_aggro(payoffs, as, bs)
    # propagate
    for i in 0:length(zs)
        if i==0
            node = ""
        else
            node = intarr_to_string(zs[1:i])
        end
        # increase count
        tree[node][2] += 1
        # increase win
        if a_win==1
            tree[node][1] += 1
        end
    end
    return tree
end


expand_leaf (generic function with 1 method)

In [49]:
function append_move_to_leaf(leaf::String, move::Int64)::String
    zs = string_to_intarr(leaf)
    append!(zs, move)
    return intarr_to_string(zs)
end

append_move_to_leaf (generic function with 1 method)

In [50]:
function is_terminal(payoffs::Array{Int64}, leaf::String)::Int64
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        #player = 1
        return lose_or_not(payoffs, bs, as)
    else
        #player = 2
        return lose_or_not(payoffs, as, bs)
    end
end

is_terminal (generic function with 1 method)

In [51]:
function get_children(payoffs::Array{Int64}, leaf::String)::Array{String}
    # get arrays
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate successors
    if player==1
        cands = legal_moves(payoffs, as, bs)
    else
        cands = legal_moves(payoffs, bs, as)
    end
    return [append_move_to_leaf(leaf, c) for c in cands]
end

get_children (generic function with 1 method)

In [52]:
function sample_next_node(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::String
    # assumes that all children are added
    children = get_children(payoffs, leaf)
    counts = [tree[c][2] for c in children]
    wins = [tree[c][1] for c in children]
    if mod(length(leaf), 2)==1
        wins = counts .- wins
    end
    scores = (wins .+ 1)./(counts .+ 1)
    return children[rand(findall(scores .== maximum(scores)))]
end

sample_next_node (generic function with 1 method)

In [53]:
function random_leaf(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String="")::String
    node = leaf
    while tree[node][2] > 1
        node = sample_next_node(payoffs, tree, node)
    end
    return node
end

random_leaf (generic function with 2 methods)

In [54]:
function display_leaf(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)
    println("Leaf:")
    as, bs = split_arrs(string_to_intarr(leaf))
    display_game(payoffs, as, bs)
    print("Stats: ")
    print(tree[leaf][1])
    print("/")
    println(tree[leaf][2])
end

function display_path(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)
    for i in 0:length(leaf)
        if i==0
            display_leaf(payoffs, tree, "")
        else
            display_leaf(payoffs, tree, leaf[1:i])
        end
        println()
    end
end

display_path (generic function with 1 method)

In [55]:
tree = Dict("" => [0,0])
# add the first node
tree = expand_leaf(payoffs, tree, "")
print(tree)

Dict("Z" => [0, 0], "`" => [0, 0], "1" => [0, 0], "!" => [0, 0], "C" => [0, 0], "2" => [0, 0], "P" => [0, 0], "<" => [0, 0], "=" => [0, 0], "/" => [0, 0], "'" => [0, 0], "." => [0, 0], "d" => [0, 0], "#" => [0, 0], "Y" => [0, 0], "U" => [0, 0], "-" => [0, 0], "\\" => [0, 0], "*" => [0, 0], "+" => [0, 0], ")" => [0, 0], "K" => [0, 0], "V" => [0, 0], "b" => [0, 0], "5" => [0, 0], "]" => [0, 0], "(" => [0, 0], "H" => [0, 0], ":" => [0, 0], "%" => [0, 0], "S" => [0, 0], "9" => [0, 0], "F" => [0, 0], "f" => [0, 0], "W" => [0, 0], "T" => [0, 0], "c" => [0, 0], ">" => [0, 0], "e" => [0, 0], "6" => [0, 0], "L" => [0, 0], "O" => [0, 0], "," => [0, 0], "B" => [0, 0], "M" => [1, 1], "[" => [0, 0], "?" => [0, 0], "i" => [0, 0], "g" => [0, 0], "X" => [0, 0], "7" => [0, 0], "D" => [0, 0], "_" => [0, 0], "8" => [0, 0], "0" => [0, 0], ";" => [0, 0], "4" => [0, 0], "Q" => [0, 0], "a" => [0, 0], "h" => [0, 0], "&" => [0, 0], "N" => [0, 0], "^" => [0, 0], "\$" => [0, 0], "A" => [0, 0], "j" => [0, 0], "G"

In [56]:
for iter in 1:1000
    # get a node with no count, then expand it
    node = ""
    while tree[node][2] > 1
        node = sample_next_node(payoffs, tree, node)
    end
    print(node)
    print(" ")
    tree = expand_leaf(payoffs, tree, node)
end

 C \ Q % / E b 6 $ Z > g R - 2 _ 7 3 . 4 # P N , c \ " 9 i ) F : N D X O c 8 U j b 2 = G ( : H E 1 Q f ] ? K i O S 0 U I ; ^ % A E' M 7 ! e h A I 9 Z a ! V h d d * W + [ 5 B d; H dO < 9F Y J @ T \" " = =R ` 2Z + a "Q J & & ` 2< i: [ ' ( V i) [* [+ i+ B (3 ($ (M (W (R iP id i& [P [6 [& 2? 2# d^ d] i` i. i6 i2 i] i' iS (+ "g 9# 9J Ea iB iG [g \0 == =9 =F =` =$ iS d/ dF 2F iS5 H3 BJ =L 9M U$ IH &M &" %7 %L %N %1 %a %< %T %8 %* %A %F () a7 a/ ZE M- hf `* c# cJ cc c? cB c( cV c6 c3 c; cD cH c^ c$ c: cb cX c+ cX cT cA c) c` cP c] cg A; Nh Vc VZ V! VH V_ OF Oi :R +, !M !@ !V bK 7B 74 !O !$ !? dg dO dL d& d6 d2 d< dC d0 !E !/ !* !1 !j !` dE Q, J" Ji !B [Q c@ i* %+ cQ cE d+ VW !) ca c= 23 2[ i3 cA cX4 cf c9 cG cM cI c@ cL c1 cU cb cK cS c" c/ c* cW c, c< cR [S [: \` cR c5 &g %^ %) OB () (7 d8 d4 d@ d* d" J. JP a> 7O 7C E. 2" "I =X =0 d! d% ce c% cO c! c\ cg c[ c. c9 c' cY c' c" c] c@V c- c, c9$ cd dK c0 i" %C !3 !R c@Q c4 (a (8 [a [! =/ =@ =M =N =? =[ c7 c` cF !S !I !a !a c`Q [X de dJ dV dM dI 

In [65]:
leaf = random_leaf(payoffs, tree, "")
display_path(payoffs, tree, leaf)

Leaf:
Stats: 438/1001

Leaf:
212221 {0}. 
Stats: 5/9

Leaf:
212221 {0}. 111142 {0}. 
Stats: 0/0

